In [ ]:
import obspy
import obspy.clients.iris
import os
import seispy

GOOGLE_DRIVE = os.environ['GOOGLE_DRIVE']
DB           = os.path.join(
    GOOGLE_DRIVE,
    'malcolm.white@usc.edu',
    'data',
    'events',
    'malcolmw',
    'SJFZ_catalog_2008-2016.h5'
)

In [ ]:
db = seispy.pandas.catalog.Catalog(
    DB, fmt='hdf5', tables=['sitechan']
)

In [ ]:
def download_data(network, station, channel, starttime, endtime):
    channel, location = channel[:3], channel[3:]
    client     = obspy.clients.iris.Client()
    starttime -= 86400
    while starttime < endtime:
        starttime += 86400
        outdir = os.path.join(
                GOOGLE_DRIVE, 
                'malcolm.white@usc.edu',
                'data',
                'wfs',
                network,
                str(starttime.year),
                f'{starttime.julday:03d}'
            )
        os.makedirs(outdir, exist_ok=True)
        try:
            path = os.path.join(outdir, f'{network}.{station}.{location if location != "" else "__"}.{channel}.{starttime.year}.{starttime.julday:03d}')
            if os.path.isfile(path):
                continue
            print(f'Requesting data file {path}')
            client.timeseries(
                network, station, location, channel, starttime, starttime+86400,
                filename=path
            )
        except Exception as exc:
            print(exc)
            pass

In [ ]:
CHANNELS = ('HHZ', 'EHZ', 'BHZ')
# NETWORK  = 'AZ'
# STATIONS = ('BZN','CRY','FRD','GARR','HSSP','PFO','RDM','RRSP','SETM','SGBF','SND','TMSP','TRAN','TRO','WMC')

NETWORK  = 'YN'
STATIONS = ('ALCY', 'BALD', 'BB04', 'BCCC', 'DWRPT', 'FOST', 'GVAR1', 'JORD', 'KLCI', 'PRAN', 'PSPR', 'RHIL', 'SFTR', 'SGBF0', 'SJR', 'SROS', 'TR01', 'TR04', 'WRDG')

# NETWORK  = 'PB'
# STATIONS = ('B081', 'B082A', 'B086A', 'B087', 'B093')

In [ ]:
df = db['sitechan'][db['sitechan']['sta'].isin(STATIONS)]
df = df[
     (df['chan'].str[1:3] == 'HZ')
    &(df['ondate'] < 2013070)
    &(
         (df['offdate'] == -1)
        |(df['offdate'] > 2013072)
    )
]

starttime = obspy.UTCDateTime('2013070')
endtime   = obspy.UTCDateTime('2013072')
for sta in df['sta'].unique():
    for chan in CHANNELS:
        if chan not in df.set_index('sta').loc[[sta], 'chan'].unique():
            continue
        else:
            download_data(NETWORK, sta, chan, starttime, endtime)
            break

In [ ]:
# ####################################################
# NETWORK   = 'AZ'
# STATION   = 'TRO'
# LOCATION  = ''
# CHANNEL   = 'HHZ'
# STARTTIME = obspy.UTCDateTime('2012-03-23T00:00:00')
# ENDTIME   = obspy.UTCDateTime('2017-01-01T00:00:00')
# ####################################################

# ####################################################
# NETWORK   = 'PB'
# STATION   = 'B086A'
# LOCATION  = ''
# CHANNEL   = 'HHZ'
# STARTTIME = obspy.UTCDateTime('2013-02-13T00:00:00')
# ENDTIME   = obspy.UTCDateTime('2017-01-01T00:00:00')
# ####################################################

####################################################
NETWORK   = 'YN'
STATION   = 'ALCY'
LOCATION  = ''
CHANNEL   = 'EHZ'
STARTTIME = obspy.UTCDateTime('2012-03-22T00:00:00')
ENDTIME   = obspy.UTCDateTime('2017-01-01T00:00:00')
####################################################

####################################################
# NETWORK   = 'YN'
# STATION   = 'SROS'
# LOCATION  = ''
# CHANNEL   = 'EHZ'
# STARTTIME = obspy.UTCDateTime('2013060T00:00:00')
# ENDTIME   = obspy.UTCDateTime('2017-01-01T00:00:00')
####################################################

In [ ]:
starttime = STARTTIME
network   = NETWORK
station   = STATION
location  = LOCATION
channel   = CHANNEL
client    = obspy.clients.iris.Client()

while starttime < ENDTIME:
    starttime += 86400
    outdir = os.path.join(
            GOOGLE_DRIVE, 
            'malcolm.white@usc.edu',
            'data',
            'wfs',
            network,
            str(starttime.year),
            f'{starttime.julday:03d}'
        )
    os.makedirs(outdir, exist_ok=True)
    try:
        path = os.path.join(outdir, f'{network}.{station}.{location if location != "" else "__"}.{channel}.{starttime.year}.{starttime.julday:03d}')
        if os.path.isfile(path):
            continue
        print(f'Requesting data file {path}')
        client.timeseries(
            network, station, location, channel, starttime, starttime+86400,
            filename=path
        )
    except Exception as exc:
        pass

In [ ]:
client = obspy.clients.iris.Client()

epoch0 = pd.to_datetime('2011-11-08').timestamp()
dfwf = db['wfdisc'][
         (db['wfdisc']['sta'].str[:3] == 'TR0')
        &(db['wfdisc']['chan'].str[2] == 'Z')
        &(db['wfdisc']['endtime'] > pd.to_datetime('2011-11-08').timestamp())
    ]
days = []
for epoch in df_ev[df_ev['time'] > epoch0].sort_values('time')['time']:
    for _, wf in dfwf[
         (dfwf['time'] < epoch)
        &(dfwf['endtime'] > epoch)
    ].iterrows():
        t0     = obspy.UTCDateTime(epoch)
        outdir = os.path.join(
            GOOGLE_DRIVE, 
            'malcolm.white@usc.edu',
            'data',
            'wfs',
            'YN',
            str(t0.year),
            f'{t0.julday:03d}'
        )
        if (t0.year, t0.julday, wf['sta']) not in days:
            days.append((t0.year, t0.julday, wf['sta']))
            t0.hour, t0.minute, t0.second, t0.microsecond = 0, 0, 0, 0
            os.makedirs(outdir, exist_ok=True)
            try:
                print(f'Downloading data for {wf.sta} {t0.year}/{t0.julday:03d}')
                client.timeseries(
                    'YN', wf['sta'], '', 'HHZ', t0, t0+86400,
                    filename=os.path.join(outdir, f'YN.{wf.sta}.HHZ.{t0.year}.{t0.julday:03d}')
                )
            except Exception as exc:
                print(exc)